# Factor Graph Python wrapper for OpenGM

https://github.com/opengm/opengm

In [39]:
import opengm
import numpy as np
from operator import itemgetter

import sys
import logging
logging.basicConfig(level=logging.DEBUG,
                    format='[%(levelname)s] (%(threadName)-10s) %(message)s',
                    stream=sys.stdout 
)

class FactorGraph(object):
    
    def __init__(self, variables, operator='multiplier'):
        """
        Factor Graph Class
        :param variables: python dictionary mapping graph variable names
                          to dimensionality.
                          For example, to define 2 binary variables:
                          {
                             'A': 2,
                             'B': 2
                          }
        :param operator: Factor graph operator, leave as 'multiplier'
        """
        assert isinstance(variables, dict)
        
        self.var_names = [vn for vn,_ in variables.items()]
        dimensionality = [variables[vn] for vn in self.var_names]

        # create factor graph with opengm
        self.gm = opengm.graphicalModel(dimensionality, 
                                        operator=operator)
        
        self.inference = None
        
    
    def add_factor_function(self, variables, probabilities):
        """
        Add a factor function to the factor graph
        
        :param variables: variables to connect in the factor graph
         NOTE: Variables must be specified in the same order as
               the initialization function of the Factor Graph
               
        :probabilities: Probability table for factor function
        """
        
        # convert probability list to np.array
        if not isinstance(probabilities, np.ndarray):
            probabilities = np.array(probabilities)
        
        # if a single variable is specified,
        # convert it to a list
        if not isinstance(variables, list):
            variables = [variables]
        
        # convert variable names to indices
        variables = [self.var_names.index(v) for v in variables]
        
        # add factor function to graph
        self.gm.addFactor(self.gm.addFunction(probabilities),
                          variables)
        
        # reset the inference
        self.inference = None
    
    def infer(self):
        """
        Run inference on the defined factor graph
        """
        self.inference = opengm.inference.BeliefPropagation(self.gm, accumulator='maximizer')
        self.inference.infer()
        
    def get_argmax(self):
        """
        Return index of state with maximum probability
        """
        if not self.inference:
            self.infer()
            
        argmax = self.inference.arg()
        
        return dict((vn, argmax[i]) for i, vn in enumerate(self.var_names))
    
    def get_marginals(self, marginal_vars):
        """
        Get marginal probabilities for specified variables
        """
        if not isinstance(marginal_vars, list):
            marginal_vars = [marginal_vars]
        
        if not self.inference:
            self.infer()
        
        marginal_probabilities =  self.inference.marginals(range(len(self.var_names)))
        marginals_ret = {}
        for v in marginal_vars:
            i = self.var_names.index(v)
            marginals_v = marginal_probabilities[i]
            marginals_v /= np.sum(marginals_v)
            marginals_ret[v] = marginals_v
            
        return marginals_ret

# Task 2 

<img src="./task2.png" width="500" height="300" />
<img src="./priors_task2.png" width="300" height="200" />

In [40]:
# create factor graph representation, i.e., add variables

# Define Factor Graph
m = FactorGraph({'S1':2, 'E1':2})
m.add_factor_function('S1', [0.9, 0.1])           # f(S1)
m.add_factor_function(['S1', 'E1'], [[0, 0.2],    # g(S1,E1)
                                     [0, 0.5]])
# Run Inference
m.infer()

# Get the marginal probabilities
marginal_S1 = m.get_marginals('S1')

# Get the argmax
argmax = m.get_argmax()


# Print the information
print("Belief S1=0 : %f" % marginal_S1['S1'][0])
print("Belief S1=1 : %f" % marginal_S1['S1'][1])
print("Therefore, S1 is in state: %d" % argmax['S1'])

Belief S1=0 : 0.782609
Belief S1=1 : 0.217391
Therefore, S1 is in state: 0


# Task 3

# Build Factor Graph model based on task description
### \# stages = 11
### \# events = 5
### \# Total Timesteps = 9, hence 9 inferences in total

### Sequence of observed events: scan -> login -> sensitive_uri -> sensitive_uri -> sensitive_uri -> new_kernel_module -> dns_tunneling -> dns_tunneling -> dns_tunneling

<table>
<tr><th><center> Security Stages </center></th><th> <center> Security Events </center> </th></tr>
<tr><td><table></table>


| Notation     	| Stage                     	|
|--------------	|---------------------------	|
| $ \sigma_1$  	| Benign                    	|
| $ \sigma_2$  	| Discovery                 	|
| $ \sigma_3$  	| Access                    	|
| $ \sigma_4$  	| Lateral Movement          	|
| $ \sigma_5$  	| Privilege Escalation      	|
| $ \sigma_6$  	| Persistence               	|
| $ \sigma_7$  	| Defense Evasion           	|
| $ \sigma_8$  	| Collection                	|
| $ \sigma_9$  	| Exfiltration              	|
| $ \sigma_{10}$ 	| Command and Control       	|
| $ \sigma_{11}$ 	| Vulnerable Code Execution 	|

</td><td>

| Notation     	| Events                     	|
|--------------	|---------------------------	|
| $ \epsilon_1$  	|  Scan                    	|
| $ \epsilon_2$  	|  Login                 	|
| $ \epsilon_3$  	|  Sensitive  URI            	|
| $ \epsilon_4$  	| New Kernel Module       	|
| $ \epsilon_5$  	|  DNS Tunneling         	|

</td></tr> </table>

In [41]:
ATTACK_STATES_MAP = {
    'benign': 1,
    'discovery': 2,
    'access': 3,
    'lateral_movement': 4,
    'privilege_escalation': 5,
    'persistence': 6,
    'defense_evasion': 7,
    'collection': 8,
    'exfiltration': 9,
    'command_control': 10,
    'execution': 11
}

ATTACKS = ['benign', 'discovery', 'access', 'lateral_movement', 'privilege_escalation', 'persistence', 'defense_evasion', 'collection', 'exfiltration', 'command_control', 'execution']

### Action Probabilities

| Stage Name            | No-Op Action | Monitor Action    | Stop Attack Action      |
|-----------------------|--------------|-------------------|-------------------------|
| Benign                | 1.00         | 0.00              | 0.00                    |
| Discovery             | 0.61         | 0.39              | 0.00                    |
| Access                | 0.69         | 0.31              | 0.00                    |
| Lateral Movement      | 0.09         | 0.84              | 0.07                    |
| Privilege Escalation  | 0.20         | 0.63              | 0.17                    |
| Persistence           | 0.00         | 0.70              | 0.30                    |
| Defense Evasion       | 0.00         | 0.07              | 0.93                    |
| Collection            | 0.00         | 0.10              | 0.90                    |
| Exfiltration          | 0.00         | 0.00              | 1.00                    |
| Command and Control   | 0.00         | 0.00              | 1.00                    |
| Execution             | 0.00         | 0.00              | 1.00                    |

In [63]:
ACTIONS = {
    # each value in an actions' vector corresponds to an attack stage
    'NO-OP':   [1.,   0.61, 0.69, 0.09, 0.2 , 0. ,  0.,   0.,   0. ,  0. ,  0.  ],
    'MONITOR': [0.  , 0.39, 0.31 ,0.84, 0.63, 0.7,  0.07 ,0.1 , 0. ,  0. ,  0.  ],
    'STOP':    [0.  , 0.,   0.  , 0.07, 0.17, 0.3,  0.93 ,0.9 , 1. ,  1. ,  1.  ]
}

# As an example, model at timestep **t=1**
#### at t = 1, $ E_1 = \text{scan} $

<img src="1.PNG" />

[//]: # (# Problem with the Naive Way)

[//]: # (#### You would need to pass the complete distribution for each factor function.) 
[//]: # (#### Size of the factor function would be equal to )

[//]: # (## $$ \prod_{V=1}^{V=\#\text{Variables connected to factor function}} (\#possible states) $$)



# $f_s$ = 
<img src="./mat_naive.png" width="600" height="400" />

## Because you have observed the sequence, you only have to consider the states corresponding to that event
[//]: # (#### However, you can do faster inference by leveraging the fact that observed events are constants in the factor graph, and you can directly pick the slice of the tensor that you need to do inference)

# $f_1$ = 
<img src="./mat_smart.png" width="600" height="400" />

<img src="./1_new.png" width=50 height=100/>

[//]: # (# Factorg Graph)
[//]: # (<img src="./fg_F_s.png" width="1000" height="800" />)

[//]: # (#   F$_s$ = \begin{bmatrix} )
[//]: # (#   p_{\sigma_1,\epsilon_1} & p_{\sigma_2,\epsilon_1} & p_{\sigma_3,\epsilon_1} & p_{\sigma_4,\epsilon_1} & p_{\sigma_5,\epsilon_1} &  p_{\sigma_6,\epsilon_1} & p_{\sigma_7,\epsilon_1} & p_{\sigma_8,\epsilon_1} & p_{\sigma_9,\epsilon_1} & p_{\sigma_{10},\epsilon_1} & p_{\sigma_{11},\epsilon_1}\\)
[//]: # (#         p_{\sigma_1,\epsilon_2} & p_{\sigma_2,\epsilon_2} & p_{\sigma_3,\epsilon_2} & p_{\sigma_4,\epsilon_2} & p_{\sigma_5,\epsilon_2} &  p_{\sigma_6,\epsilon_2} & p_{\sigma_7,\epsilon_2} & p_{\sigma_8,\epsilon_2} & p_{\sigma_9,\epsilon_2} & p_{\sigma_{10},\epsilon_2} & p_{\sigma_{11},\epsilon_2} \\)
[//]: # (#             p_{\sigma_1,\epsilon_3} & p_{\sigma_2,\epsilon_3} & p_{\sigma_3,\epsilon_3} & p_{\sigma_4,\epsilon_3} & p_{\sigma_5,\epsilon_3} &  p_{\sigma_6,\epsilon_3} & p_{\sigma_7,\epsilon_3} & p_{\sigma_8,\epsilon_3} & p_{\sigma_9,\epsilon_3} & p_{\sigma_{10},\epsilon_3} & p_{\sigma_{11},\epsilon_5}\\)
[//]: # (#                 p_{\sigma_1,\epsilon_4} & p_{\sigma_2,\epsilon_4} & p_{\sigma_3,\epsilon_4} & p_{\sigma_4,\epsilon_4} & p_{\sigma_5,\epsilon_4} &  p_{\sigma_6,\epsilon_4} & p_{\sigma_7,\epsilon_4} & p_{\sigma_8,\epsilon_4} & p_{\sigma_9,\epsilon_4} & p_{\sigma_{10},\epsilon_4} & p_{\sigma_{11},\epsilon_5}\\)
[//]: # (#                     p_{\sigma_1,\epsilon_5} & p_{\sigma_2,\epsilon_5} & p_{\sigma_3,\epsilon_5} & p_{\sigma_4,\epsilon_5} & p_{\sigma_5,\epsilon_5} &  p_{\sigma_6,\epsilon_5} & p_{\sigma_7,\epsilon_5} & p_{\sigma_8,\epsilon_5} & p_{\sigma_9,\epsilon_5} & p_{\sigma_{10},\epsilon_5} & p_{\sigma_{11},\epsilon_5}\\)
[//]: # (#   \end{bmatrix})

# Factor Function Definition:
<img src="pq.png" />

# Recap on significance
### The P-value answers the following question: 
### What is the probability of the observed test statistic or one more extreme when H0 is true? 

<img src="./significance.png" width="600" height="400" />

### Belief is defined as q(1-p),
#### where p is significance
#### and q is Probability in past attacks 


In [62]:
def get_prob(stages, p, q):
    assert len(p) == len(q) == len(stages)
    prob = np.zeros(11)
    for i in range(len(p)):
        # -1 as the indexing in Python lists begin from 0
        stage_idx = ATTACK_STATES_MAP[stages[i]] - 1 
        prob[stage_idx] = q[i] * (1 - p[i])
    # convert to an 1 x 11 matrix
    return np.array(prob)

def print_state_belief(mValues):
    mValues = list(mValues)
    print("Beliefs", list(zip(ATTACKS,mValues[0:len(ATTACKS)])))

# Solution to model at timestep, t = 1

In [61]:
# Lets create the factor graph in our solver corresponding to the model defined above
# Add S1 to the model. Note the dimensionality of S1 is 11
m = FactorGraph({'S1': 11})

# Add the correspoding factor function f_1 
S1Stagesf1 = ['discovery', 'benign']
m.add_factor_function('S1', get_prob(S1Stagesf1, [0.04, 0.47], [0.5, 0.5]))

# Run inference on the model which will help estimate the hidden state
m.infer()

marginal_S1 = m.get_marginals('S1')
# print state values
# print marginal_S1['S1']
print_state_belief(marginal_S1['S1'])

# Get the attack stage
# To obtain the stage with max porbability, 
# we apply get_argmax() to get argmax, 
# which means the index of max probability stage

argmax = m.get_argmax()
print(argmax)
print('Attack stage is %s' %  ATTACKS[argmax['S1']])


# Determine the appropriate action associated with the attack stage

# to determine the action to be taken, we look at the probability values
# for the discovery stage for all posible actions, and pick the action
# with the maximum probability
idx = argmax['S1'] # 
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print("Action Probabilities", action_probabilities)
max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print("Selected Action is %s" % max_action)

('Beliefs', [('benign', 0.35570469798657717), ('discovery', 0.6442953020134228), ('access', 0.0), ('lateral_movement', 0.0), ('privilege_escalation', 0.0), ('persistence', 0.0), ('defense_evasion', 0.0), ('collection', 0.0), ('exfiltration', 0.0), ('command_control', 0.0), ('execution', 0.0)])
{'S1': 1}
Attack stage is discovery
('Action Probabilities', [('STOP', 0.0), ('MONITOR', 0.39), ('NO-OP', 0.61)])
Selected Action is NO-OP


# # Factor Graph at t = 2
### $ E_1 = \text{scan}$
### $ E_2 = \text{login}$

<img src="./t_2.png" />

### After making observations


<img src="./t_2_impl.png" />

In [60]:
m = FactorGraph({'S1': 11,'S2':11})
S1Stagesf1 = ['discovery', 'benign']
m.add_factor_function('S1', get_prob(S1Stagesf1, [0.04, 0.47], [0.5, 0.5]))
S2Stagesf2 = ['benign']
m.add_factor_function('S2', get_prob(S2Stagesf2, [0.01], [0.5]))
m.infer()

marginal_S2 = m.get_marginals('S2')
print_state_belief(marginal_S2['S2'])

argmax = m.get_argmax()
print('Attack stage is %s' % ATTACKS[argmax['S2']])

idx = argmax['S2']
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print("Action Probabilities", action_probabilities)

max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print("Selected Action is %s" % max_action)

('Beliefs', [('benign', 1.0), ('discovery', 0.0), ('access', 0.0), ('lateral_movement', 0.0), ('privilege_escalation', 0.0), ('persistence', 0.0), ('defense_evasion', 0.0), ('collection', 0.0), ('exfiltration', 0.0), ('command_control', 0.0), ('execution', 0.0)])
Attack stage is benign
('Action Probabilities', [('STOP', 0.0), ('MONITOR', 0.0), ('NO-OP', 1.0)])
Selected Action is NO-OP


# Factor Graph at t = 5 

In [59]:
m = FactorGraph({'S1': 11,'S2':11,'S3':11,'S4':11,'S5':11})
S1Stagesf1 = ['discovery', 'benign']
m.add_factor_function('S1', get_prob(S1Stagesf1, [0.04, 0.47], [0.5, 0.5]))
S2Stagesf2 = ['benign']
m.add_factor_function('S2', get_prob(S2Stagesf2, [0.01], [0.5]))
S3Stagesf3 = ['privilege_escalation', 'benign']
m.add_factor_function('S3', get_prob(S3Stagesf3, [0.02, 0.02], [0.1, 0.1]))
S4Stagesf4 = ['privilege_escalation', 'benign']
m.add_factor_function('S4', get_prob(S4Stagesf4, [0.02, 0.02], [0.1, 0.1]))
S5Stagesf5 = ['privilege_escalation', 'benign']
m.add_factor_function('S5', get_prob(S5Stagesf5, [0.02, 0.02], [0.1, 0.1]))

# Need to add factor function r as privelage_escalation is repeated thrice 
# and according to the table, we need to add corresponding factor function r
S5StagesR = ['privilege_escalation']
m.add_factor_function('S5', get_prob(S5StagesR, [0.05], [0.15])) #r
m.infer()


<img src="./factorR.png" />

In [33]:
print("Done")

Done
